In [1]:
#custom stuff
print("ok")

ok


In [2]:
import numpy as np
import tensorflow as tf 

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,Subtract,Reshape
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Conv2D,MaxPooling2D,Input,Lambda,GlobalMaxPooling2D
from keras.regularizers import l2
from keras import backend as K
from keras.applications.vgg16 import VGG16
from skimage.io import imsave

from matplotlib.pyplot import imread
from skimage.transform import rescale, resize
import os
from keras.models import load_model
from sklearn.model_selection import train_test_split

print("loaded libs.")

loaded libs.


In [3]:
# Dataset parameters 
dataset_path = '../../datasets/VisualPhish/'
reshape_size = [224,224,3]
phishing_test_size = 0.4
num_targets = 24

# Model parameters
input_shape = [224,224,3]
margin = 2.2
new_conv_params = [3,3,512]

# Training parameters
start_lr = 0.00002
output_dir = './'
saved_model_name = 'model'
save_interval = 2000
batch_size = 32 
n_iter = 1000 #50000
lr_interval = 300

# !find datasets/trusted_list/ -name *.png | wc -l
# !find datasets/phishing/ -name *.png | wc -l
#2031
#3821

# Load dataset:
    - Load training screenshots per website
    - Load Phishing screenshots per website 

In [4]:

def read_imgs_per_website(data_path,targets,imgs_num,reshape_size,start_target_count):
    all_imgs = np.zeros(shape=[imgs_num,224,224,3])
    all_labels = np.zeros(shape=[imgs_num,1])
    
    all_file_names = []
    targets_list = targets.splitlines()
    count = 0
    for i in range(0,len(targets_list)):
        target_path = data_path + targets_list[i]
        print("[{}]loading:{}".format(i,target_path))
        file_names = sorted(os.listdir(target_path))
        for j in range(0,len(file_names)):
            try:
                img = imread(target_path+'/'+file_names[j])
                img = img[:,:,0:3]
                all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                all_labels[count,:] = i + start_target_count
                all_file_names.append(file_names[j])
                count = count + 1
            except:
                #some images were saved with a wrong extensions 
                try:
                    img = imread(target_path+'/'+file_names[j],format='jpeg')
                    img = img[:,:,0:3]
                    all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                    all_labels[count,:] = i + start_target_count
                    all_file_names.append(file_names[j])
                    count = count + 1
                except:
                    print('failed at:')
                    print('***')
                    print(file_names[j])
                    break 
    return all_imgs,all_labels,all_file_names



In [5]:
# Read images legit (train)
data_path = dataset_path + 'dataset_new_phishing_1366_768_trusted/'
targets_file = open(data_path+'targets.txt', "r")
targets = targets_file.read()
imgs_num = 1975     #9363
all_imgs_train,all_labels_train,all_file_names_train = read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

# Read images phishing
data_path = dataset_path + 'dataset_new_phishing_1366_768_phish/'
targets_file = open(data_path+'targets.txt', "r")
targets = targets_file.read()
imgs_num = 1095     #1195
all_imgs_test,all_labels_test,all_file_names_test = read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

X_train_legit = all_imgs_train
y_train_legit = all_labels_train

# Split phishing to training and test 
idx = np.arange(all_imgs_test.shape[0])
X_test_phish, X_train_phish, y_test_phish, y_train_phish,idx_test,idx_train = train_test_split(all_imgs_test, all_labels_test,idx, test_size=phishing_test_size)
np.save(output_dir+'test_idx',idx_test)
np.save(output_dir+'train_idx',idx_train)
print("load complete.")

[0]loading:../../datasets/VisualPhish/dataset_new_phishing_1366_768_trusted/adobe.com
[1]loading:../../datasets/VisualPhish/dataset_new_phishing_1366_768_trusted/github.com
[2]loading:../../datasets/VisualPhish/dataset_new_phishing_1366_768_trusted/player.vimeo.com
[3]loading:../../datasets/VisualPhish/dataset_new_phishing_1366_768_trusted/wordpress.com
[4]loading:../../datasets/VisualPhish/dataset_new_phishing_1366_768_trusted/amazon.com
[5]loading:../../datasets/VisualPhish/dataset_new_phishing_1366_768_trusted/google.com
[6]loading:../../datasets/VisualPhish/dataset_new_phishing_1366_768_trusted/play.google.com
[7]loading:../../datasets/VisualPhish/dataset_new_phishing_1366_768_trusted/wordpress.org
[8]loading:../../datasets/VisualPhish/dataset_new_phishing_1366_768_trusted/apache.org
[9]loading:../../datasets/VisualPhish/dataset_new_phishing_1366_768_trusted/gravatar.com
[10]loading:../../datasets/VisualPhish/dataset_new_phishing_1366_768_trusted/yahoo.com
[11]loading:../../dataset

# Create Model

In [6]:
def define_triplet_network(input_shape, new_conv_params):
    
    # Input_shape: shape of input images
    # new_conv_params: dimension of the new convolution layer [spatial1,spatial2,channels] 
    
    # Define the tensors for the three input images
    anchor_input = Input(input_shape)
    positive_input = Input(input_shape)
    negative_input = Input(input_shape)
    
    # Use VGG as a base model 
    base_model = VGG16(weights='imagenet',  input_shape=input_shape, include_top=False)

    x = base_model.output
    x = Conv2D(new_conv_params[2],(new_conv_params[0],new_conv_params[1]),activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(2e-4)) (x)
    x = GlobalAveragePooling2D() (x)
    model = Model(inputs=base_model.input, outputs=x)

    # Generate the encodings (feature vectors) for the two images
    encoded_a = model(anchor_input)
    encoded_p = model(positive_input)
    encoded_n = model(negative_input)
    
    mean_layer = Lambda(lambda x: K.mean(x,axis=1))
    
    square_diff_layer = Lambda(lambda tensors:K.square(tensors[0] - tensors[1]))
    square_diff_pos = square_diff_layer([encoded_a,encoded_p])
    square_diff_neg = square_diff_layer([encoded_a,encoded_n])
    
    square_diff_pos_l2 = mean_layer(square_diff_pos)
    square_diff_neg_l2 = mean_layer(square_diff_neg)
    
    # Add a diff layer
    diff = Subtract()([square_diff_pos_l2, square_diff_neg_l2])
    diff = Reshape((1,)) (diff)

    # Connect the inputs with the outputs
    triplet_net = Model(inputs=[anchor_input,positive_input,negative_input],outputs=diff)
    
    # return the model
    return triplet_net

In [7]:
# Define the triplet loss
# Create and compile model 

def custom_loss(margin):
    def loss(y_true,y_pred):
        loss_value = K.maximum(y_true, margin + y_pred)
        loss_value = K.mean(loss_value,axis=0)
        return loss_value
    return loss
def loss(y_true,y_pred):
    loss_value = K.maximum(y_true, margin + y_pred)
    loss_value = K.mean(loss_value,axis=0)
    return loss_value


model = define_triplet_network(input_shape, new_conv_params)
model.summary()

from keras import optimizers
optimizer = optimizers.Adam(lr = start_lr)
model.compile(loss=custom_loss(margin),optimizer=optimizer)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 512)          17074496    input_1[0][0]                    
                                                                 input_2[0][0]              

# Triplet Sampling

In [8]:
# Order random phishing arrays per website (from 0 to 155 target)

def order_random_array(orig_arr,y_orig_arr,targets):
    sorted_arr = np.zeros(orig_arr.shape)
    y_sorted_arr = np.zeros(y_orig_arr.shape)
    count = 0
    for i in range(0,targets):
        for j in range(0,orig_arr.shape[0]):
            if y_orig_arr[j] == i:
                sorted_arr[count,:,:,:] = orig_arr[j,:,:,:]
                y_sorted_arr[count,:] = i
                count = count + 1
    return sorted_arr,y_sorted_arr 

X_test_phish,y_test_phish = order_random_array(X_test_phish,y_test_phish,num_targets)
X_train_phish,y_train_phish = order_random_array(X_train_phish,y_train_phish,num_targets)


# Store the start and end of each target in the phishing set (used later in triplet sampling)
# Not all targets might be in the phishing set 
def targets_start_end(num_target,labels):
    prev_target = labels[0]
    start_end_each_target = np.zeros((num_target,2))
    start_end_each_target[0,0] = labels[0]
    if not labels[0] == 0:
        start_end_each_target[0,0] = -1
        start_end_each_target[0,1] = -1
    count_target = 0
    for i in range(1,labels.shape[0]):
        if not labels[i] == prev_target:
            start_end_each_target[int(labels[i-1]),1] = int(i-1)
            start_end_each_target[int(labels[i]),0] = int(i)
            prev_target = labels[i]
    start_end_each_target[int(labels[-1]),1] = int(labels.shape[0]-1)
    
    for i in range(1,num_target):
        if start_end_each_target[i,0] == 0:
            start_end_each_target[i,0] = -1
            start_end_each_target[i,1] = -1
    return start_end_each_target

labels_start_end_train_phish = targets_start_end(num_targets,y_train_phish)
labels_start_end_test_phish = targets_start_end(num_targets,y_test_phish)


# Store the start and end of each target in the training set (used later in triplet sampling)
def all_targets_start_end(num_target,labels):
    prev_target = 0
    start_end_each_target = np.zeros((num_target,2))
    start_end_each_target[0,0] = 0
    count_target = 0
    for i in range(1,labels.shape[0]):
        if not labels[i] == prev_target:
            start_end_each_target[count_target,1] = i-1
            count_target = count_target + 1
            start_end_each_target[count_target,0] = i
            prev_target = prev_target + 1
    start_end_each_target[num_target-1,1] = labels.shape[0]-1
    return start_end_each_target

labels_start_end_train_legit = all_targets_start_end(num_targets,y_train_legit)


In [9]:
#Sample anchor, positive and negative images 
def pick_first_img_idx(labels_start_end,num_targets):
    random_target = -1
    while (random_target == -1):
        random_target = np.random.randint(low = 0,high = num_targets)
        if labels_start_end[random_target,0] == -1:
            random_target = -1
    return random_target

def pick_pos_img_idx(prob_phish,img_label):
    if np.random.uniform() > prob_phish:
        class_idx_start_end = labels_start_end_train_legit[img_label,:]
        same_idx = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
        img = X_train_legit[same_idx,:]
    else:
        if not labels_start_end_train_phish[img_label,0] == -1:
            class_idx_start_end = labels_start_end_train_phish[img_label,:]
            same_idx = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
            img = X_train_phish[same_idx,:]
        else:
            class_idx_start_end = labels_start_end_train_legit[img_label,:]
            same_idx = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
            img = X_train_legit[same_idx,:]
    return img

def pick_neg_img(anchor_idx,num_targets):
    if anchor_idx == 0:
        targets = np.arange(1,num_targets)
    elif anchor_idx == num_targets -1:
        targets = np.arange(0,num_targets-1)
    else:
        targets = np.concatenate([np.arange(0,anchor_idx),np.arange(anchor_idx+1,num_targets)])
    diff_target_idx = np.random.randint(low = 0,high = num_targets-1)
    diff_target = targets[diff_target_idx]
    
    class_idx_start_end = labels_start_end_train_legit[diff_target,:]
    lo = min(class_idx_start_end[0],class_idx_start_end[1]+1)
    hi = max(class_idx_start_end[0],class_idx_start_end[1]+1)
    idx_from_diff_target = np.random.randint(low = lo,high = hi)
    img = X_train_legit[idx_from_diff_target,:]
    
    return img,diff_target

# Sample batches

In [10]:
# Don't sample negative image from the same category as the positive image (e.g. google and google drive)
# Create clusters of same-company websites (e.g. all microsoft websites)


targets_file = open(data_path+'targets.txt', "r")
all_targets = targets_file.read()
all_targets = all_targets.splitlines()

def get_idx_of_target(target_name,all_targets):
    for i in range(0,len(all_targets)):
        if all_targets[i] == target_name:
            found_idx = i
            return found_idx
        
#targets names of parent and sub websites
target_lists = [['microsoft','ms_outlook','ms_office','ms_bing','ms_onedrive','ms_skype'],['apple','itunes','icloud'],['google','google_drive'],['alibaba','aliexpress']]

def get_associated_targets_idx(target_lists,all_targets):
    sub_target_lists_idx = []
    parents_ids = []
    for i in range(0,len(target_lists)):
        target_list = target_lists[i]
        parent_target = target_list[0]
        one_target_list = []
        parent_idx = get_idx_of_target(parent_target,all_targets)
        parents_ids.append(parent_idx)
        for child_target in target_list[1:]:
            child_idx = get_idx_of_target(child_target,all_targets)
            one_target_list.append(child_idx)
        sub_target_lists_idx.append(one_target_list)
    return parents_ids,sub_target_lists_idx 

parents_ids,sub_target_lists_idx  = get_associated_targets_idx(target_lists,all_targets)

def check_if_same_category(img_label1,img_label2):
    if_same = 0
    if img_label1 in parents_ids:
        if img_label2 in sub_target_lists_idx[parents_ids.index(img_label1)]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[0]:
        if img_label2 in sub_target_lists_idx[0] or img_label2 == parents_ids[0]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[1]:
        if img_label2 in sub_target_lists_idx[1] or img_label2 == parents_ids[1]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[2]:
        if img_label2 in sub_target_lists_idx[2] or img_label2 == parents_ids[2]:
            if_same = 1
    return if_same


In [11]:
#sample triplets
def get_batch(batch_size,num_targets):
   
    # initialize 3 empty arrays for the input image batch
    h = X_train_legit.shape[1]
    w = X_train_legit.shape[2]
    triple=[np.zeros((batch_size, h, w,3)) for i in range(3)]

    for i in range(0,batch_size):
        img_idx_pair1 = pick_first_img_idx(labels_start_end_train_legit,num_targets)
        triple[0][i,:,:,:] = X_train_legit[img_idx_pair1,:]
        img_label = int(y_train_legit[img_idx_pair1])
        
        # get image for the second: positive
        triple[1][i,:,:,:] = pick_pos_img_idx(0.15,img_label)
            
        # get image for the thrid: negative from legit
        # don't sample from the same cluster
        img_neg,label_neg = pick_neg_img(img_label,num_targets)
        while check_if_same_category(img_label,label_neg) == 1:
            img_neg,label_neg = pick_neg_img(img_label,num_targets)

        triple[2][i,:,:,:] = img_neg
          
    return triple


# Training

In [12]:
def save_keras_model(model):
    model.save(output_dir+saved_model_name+'.h5')
    print("Saved model to disk")

In [13]:
print("Starting training process!")
print("-------------------------------------")
import datetime
targets_train = np.zeros([batch_size,1])
ts1=datetime.datetime.now()
for i in range(1, n_iter):
    inputs=get_batch(batch_size,num_targets)
    loss_value=model.train_on_batch(inputs,targets_train)
    
    ts2=datetime.datetime.now()
    td = ts2-ts1
    print("\n ------------- \n")
    print("Iteration:{}. Loss: {}, total seconds:{}".format(i,loss_value,td.total_seconds()))
    
    if i % save_interval == 0:
        save_keras_model(model)
        
    if i % lr_interval ==0:
        start_lr = 0.99*start_lr
        K.set_value(model.optimizer.lr, start_lr)

save_keras_model(model)

Starting training process!
-------------------------------------

 ------------- 

Iteration:1. Loss: 2.374183416366577, total seconds:38.722628

 ------------- 

Iteration:2. Loss: 2.3508732318878174, total seconds:83.684977

 ------------- 

Iteration:3. Loss: 2.301535129547119, total seconds:128.197266

 ------------- 

Iteration:4. Loss: 2.2013702392578125, total seconds:172.256602

 ------------- 

Iteration:5. Loss: 2.0567219257354736, total seconds:215.519983

 ------------- 

Iteration:6. Loss: 1.7357285022735596, total seconds:258.497026

 ------------- 

Iteration:7. Loss: 1.4305474758148193, total seconds:301.901118

 ------------- 

Iteration:8. Loss: 0.9213442206382751, total seconds:344.778096

 ------------- 

Iteration:9. Loss: 0.7278574705123901, total seconds:387.306105

 ------------- 

Iteration:10. Loss: 0.8935000896453857, total seconds:429.928536

 ------------- 

Iteration:11. Loss: 0.6817028522491455, total seconds:472.077613

 ------------- 

Iteration:12. Los


 ------------- 

Iteration:98. Loss: 0.22311225533485413, total seconds:4242.792549

 ------------- 

Iteration:99. Loss: 0.19159872829914093, total seconds:4287.223326

 ------------- 

Iteration:100. Loss: 0.19148309528827667, total seconds:4332.155923

 ------------- 

Iteration:101. Loss: 0.20493005216121674, total seconds:4378.048146

 ------------- 

Iteration:102. Loss: 0.19125191867351532, total seconds:4423.467185

 ------------- 

Iteration:103. Loss: 0.19113637506961823, total seconds:4468.018083

 ------------- 

Iteration:104. Loss: 0.19102077186107635, total seconds:4513.922154

 ------------- 

Iteration:105. Loss: 0.24656453728675842, total seconds:4558.390758

 ------------- 

Iteration:106. Loss: 0.19078922271728516, total seconds:4602.90804

 ------------- 

Iteration:107. Loss: 0.190673366189003, total seconds:4648.676716

 ------------- 

Iteration:108. Loss: 0.19055739045143127, total seconds:4694.335897

 ------------- 

Iteration:109. Loss: 0.19044137001037598,


 ------------- 

Iteration:194. Loss: 0.18055899441242218, total seconds:8545.045583

 ------------- 

Iteration:195. Loss: 0.18044708669185638, total seconds:8588.588766

 ------------- 

Iteration:196. Loss: 0.1803351491689682, total seconds:8632.983629

 ------------- 

Iteration:197. Loss: 0.18022319674491882, total seconds:8676.752204

 ------------- 

Iteration:198. Loss: 0.18011125922203064, total seconds:8721.117383

 ------------- 

Iteration:199. Loss: 0.17999933660030365, total seconds:8764.585853

 ------------- 

Iteration:200. Loss: 0.17988738417625427, total seconds:8808.874544

 ------------- 

Iteration:201. Loss: 0.1797754466533661, total seconds:8852.879198

 ------------- 

Iteration:202. Loss: 0.1796635240316391, total seconds:8897.390671

 ------------- 

Iteration:203. Loss: 0.1795516163110733, total seconds:8941.495178

 ------------- 

Iteration:204. Loss: 0.1794397085905075, total seconds:8986.304334

 ------------- 

Iteration:205. Loss: 0.1793278157711029, 


 ------------- 

Iteration:289. Loss: 0.17076189815998077, total seconds:12751.303785

 ------------- 

Iteration:290. Loss: 0.17066845297813416, total seconds:12796.064427

 ------------- 

Iteration:291. Loss: 0.17057502269744873, total seconds:12839.729333

 ------------- 

Iteration:292. Loss: 0.1704816073179245, total seconds:12884.128885

 ------------- 

Iteration:293. Loss: 0.17038820683956146, total seconds:12928.637707

 ------------- 

Iteration:294. Loss: 0.170294851064682, total seconds:12973.493538

 ------------- 

Iteration:295. Loss: 2.680610418319702, total seconds:13017.053232

 ------------- 

Iteration:296. Loss: 0.3992357850074768, total seconds:13061.584442

 ------------- 

Iteration:297. Loss: 0.35182374715805054, total seconds:13105.722198

 ------------- 

Iteration:298. Loss: 0.1699412614107132, total seconds:13149.472173

 ------------- 

Iteration:299. Loss: 0.16985580325126648, total seconds:13193.485081

 ------------- 

Iteration:300. Loss: 0.288543105


 ------------- 

Iteration:384. Loss: 0.16326618194580078, total seconds:16935.083538

 ------------- 

Iteration:385. Loss: 0.16319020092487335, total seconds:16978.290978

 ------------- 

Iteration:386. Loss: 0.16311432421207428, total seconds:17021.546446

 ------------- 

Iteration:387. Loss: 0.16303841769695282, total seconds:17064.793863

 ------------- 

Iteration:388. Loss: 0.16296260058879852, total seconds:17107.582122

 ------------- 

Iteration:389. Loss: 0.16288678348064423, total seconds:17151.447306

 ------------- 

Iteration:390. Loss: 0.1628110259771347, total seconds:17194.991236

 ------------- 

Iteration:391. Loss: 0.16273532807826996, total seconds:17237.938888

 ------------- 

Iteration:392. Loss: 0.1626596301794052, total seconds:17281.670195

 ------------- 

Iteration:393. Loss: 0.16258396208286285, total seconds:17324.706244

 ------------- 

Iteration:394. Loss: 0.16250836849212646, total seconds:17368.513005

 ------------- 

Iteration:395. Loss: 0.1624

...


 ------------- 

Iteration:574. Loss: 0.15007255971431732, total seconds:25227.485052

 ------------- 

Iteration:575. Loss: 0.1500084400177002, total seconds:25271.074554

 ------------- 

Iteration:576. Loss: 0.14994436502456665, total seconds:25317.046159

 ------------- 

Iteration:577. Loss: 0.1498803198337555, total seconds:25361.954973

 ------------- 

Iteration:578. Loss: 0.14981628954410553, total seconds:25405.957666

 ------------- 

Iteration:579. Loss: 0.14975231885910034, total seconds:25450.148269

 ------------- 

Iteration:580. Loss: 0.14968833327293396, total seconds:25493.55497

 ------------- 

Iteration:581. Loss: 0.14962440729141235, total seconds:25536.403674

 ------------- 

Iteration:582. Loss: 0.14956051111221313, total seconds:25579.805026

 ------------- 

Iteration:583. Loss: 0.1494966447353363, total seconds:25623.568567

 ------------- 

Iteration:584. Loss: 0.14943279325962067, total seconds:25667.194323

 ------------- 

Iteration:585. Loss: 0.149369


 ------------- 

Iteration:669. Loss: 0.14419028162956238, total seconds:29380.647109

 ------------- 

Iteration:670. Loss: 0.14413024485111237, total seconds:29424.362007

 ------------- 

Iteration:671. Loss: 0.14407025277614594, total seconds:29468.10122

 ------------- 

Iteration:672. Loss: 0.1440102756023407, total seconds:29511.731437

 ------------- 

Iteration:673. Loss: 0.2156575322151184, total seconds:29555.072243

 ------------- 

Iteration:674. Loss: 0.14389045536518097, total seconds:29598.748236

 ------------- 

Iteration:675. Loss: 0.1438305675983429, total seconds:29642.370525

 ------------- 

Iteration:676. Loss: 0.1437707394361496, total seconds:29686.009435

 ------------- 

Iteration:677. Loss: 0.1437109410762787, total seconds:29730.083988

 ------------- 

Iteration:678. Loss: 0.14365117251873016, total seconds:29774.285471

 ------------- 

Iteration:679. Loss: 0.14359143376350403, total seconds:29817.946867

 ------------- 

Iteration:680. Loss: 0.14353173


 ------------- 

Iteration:764. Loss: 0.13863644003868103, total seconds:33532.683543

 ------------- 

Iteration:765. Loss: 0.1385795623064041, total seconds:33576.549296

 ------------- 

Iteration:766. Loss: 0.13852271437644958, total seconds:33620.770144

 ------------- 

Iteration:767. Loss: 0.13846589624881744, total seconds:33663.768884

 ------------- 

Iteration:768. Loss: 0.1384091079235077, total seconds:33708.512647

 ------------- 

Iteration:769. Loss: 0.13835236430168152, total seconds:33751.448947

 ------------- 

Iteration:770. Loss: 0.13829565048217773, total seconds:33794.664794

 ------------- 

Iteration:771. Loss: 0.13823896646499634, total seconds:33838.579431

 ------------- 

Iteration:772. Loss: 0.13818231225013733, total seconds:33882.556553

 ------------- 

Iteration:773. Loss: 0.1381256878376007, total seconds:33926.623296

 ------------- 

Iteration:774. Loss: 0.13806909322738647, total seconds:33969.610581

 ------------- 

Iteration:775. Loss: 0.13801


 ------------- 

Iteration:859. Loss: 0.1333756148815155, total seconds:37687.539737

 ------------- 

Iteration:860. Loss: 0.13332174718379974, total seconds:37730.77834

 ------------- 

Iteration:861. Loss: 0.13326790928840637, total seconds:37773.916314

 ------------- 

Iteration:862. Loss: 0.1332141011953354, total seconds:37817.607662

 ------------- 

Iteration:863. Loss: 0.1331603229045868, total seconds:37861.363544

 ------------- 

Iteration:864. Loss: 0.13310658931732178, total seconds:37905.022605

 ------------- 

Iteration:865. Loss: 0.13305285573005676, total seconds:37949.314265

 ------------- 

Iteration:866. Loss: 0.13299919664859772, total seconds:37992.72375

 ------------- 

Iteration:867. Loss: 0.13294552266597748, total seconds:38036.230683

 ------------- 

Iteration:868. Loss: 0.19958707690238953, total seconds:38079.460063

 ------------- 

Iteration:869. Loss: 0.13283830881118774, total seconds:38123.503916

 ------------- 

Iteration:870. Loss: 0.1327847


 ------------- 

Iteration:954. Loss: 0.12841971218585968, total seconds:41839.071008

 ------------- 

Iteration:955. Loss: 0.12836916744709015, total seconds:41883.258603

 ------------- 

Iteration:956. Loss: 0.128318652510643, total seconds:41926.413784

 ------------- 

Iteration:957. Loss: 0.12826818227767944, total seconds:41969.72181

 ------------- 

Iteration:958. Loss: 0.12821771204471588, total seconds:42012.902194

 ------------- 

Iteration:959. Loss: 0.1281672865152359, total seconds:42057.088093

 ------------- 

Iteration:960. Loss: 0.12811687588691711, total seconds:42100.825201

 ------------- 

Iteration:961. Loss: 0.12806649506092072, total seconds:42144.188569

 ------------- 

Iteration:962. Loss: 0.1280161589384079, total seconds:42187.723781

 ------------- 

Iteration:963. Loss: 0.12796583771705627, total seconds:42231.297469

 ------------- 

Iteration:964. Loss: 0.12791553139686584, total seconds:42274.780755

 ------------- 

Iteration:965. Loss: 0.1278652

# Calculate Embeddings

#### shared_model = model.layers[3]

whitelist_emb = shared_model.predict(X_train_legit,batch_size=64)
np.save(output_dir+'whitelist_emb',whitelist_emb)
np.save(output_dir+'whitelist_labels',y_train_legit )

phishing_emb = shared_model.predict(all_imgs_test,batch_size=64)
np.save(output_dir+'phishing_emb',phishing_emb)
np.save(output_dir+'phishing_labels',all_labels_test )
